In [1]:
from netCDF4 import Dataset
import numpy as np


In [2]:


predictor2file = {'airTemp':'tas.nc', 'precip':'pr.nc', 'downwardLongwave':'rlns.nc', \
                  'downwardShortwave':'rsds.nc', 'plantAvailableWater':'paw.nc'}
# where is relative humidity?
predictors = {k:Dataset(v)['var1'][...].squeeze() for k,v in predictor2file.items()}
targetFile = 'jedi_output.nc'
targetVariable = 'NPP'
target = Dataset( targetFile)[targetVariable][...].squeeze()
landSeaMask = Dataset('landsea.nc')['var1'][...].squeeze()

# an example of making annual means
precip = predictors['precip']
annualMeanPrecip = precip.reshape((-1, 12, precip.shape[1], precip.shape[2])).mean( axis=1)

In [3]:
print(np.shape(annualMeanPrecip))

(24, 51, 60)


In [4]:
print(type(predictors))

<class 'dict'>


In [5]:
airTemp = 'tas.nc'
netLongRadiantionFlux = 'rlns.nc'
shortwaveRadiation = 'rsds.nc'
precipitation = 'pr.nc'
output = 'jedi_output.nc'

landSeaMask = 'landsea.nc'
lantitude =  'latitude.nc'
longitude = 'longitude.nc'
elevation = 'elevation.nc'
moisture = 'paw.nc'
spatialMetaData = 'globe_surf.nc'

airTempDataset = Dataset(airTemp)
netLongRadiantionFluxDataSet = Dataset(netLongRadiantionFlux)
shortwaveRadiationDataset = Dataset(shortwaveRadiation)
precipitationDataSet = Dataset(precipitation)
outputDataset = Dataset(output)

landSeaMaskDataset = Dataset(landSeaMask)
lantitudeDataset = Dataset(lantitude)
longitudeDataset = Dataset(longitude)
elevationDataset = Dataset(elevation)
moistureDataset = Dataset(moisture)
spatialMetaDataset = Dataset(spatialMetaData)

airTempArray = airTempDataset['var1'][:].squeeze()
netLongRadiantionFluxArray = netLongRadiantionFluxDataSet['var1'][:].squeeze()
shortwaveRadiationArray = shortwaveRadiationDataset['var1'][:].squeeze()
precipitationArray = precipitationDataSet['var1'][:].squeeze()
NPPArray = outputDataset['NPP'][:].squeeze()
landSeaMaskArray = landSeaMaskDataset['var1'][:].squeeze()
lantitudeArray = lantitudeDataset['var1'][:].squeeze()
longitudeArray = longitudeDataset['var1'][:].squeeze()
elevationArray = elevationDataset['var1'][:].squeeze()
moistureArray = moistureDataset['var1'][:].squeeze()
spatialMetaDataArray = spatialMetaData[:]



In [6]:
print('airTempArray'+str(np.shape(airTempArray)))
print('netLongRadiantionFluxArray'+str(np.shape(netLongRadiantionFluxArray)))
print('shortwaveRadiationArray'+str(np.shape(shortwaveRadiationArray)))
print('precipitationArray'+str(np.shape(precipitationArray)))
print('NPPArray'+str(np.shape(NPPArray)))
print('landSeaMaskArray'+str(np.shape(landSeaMaskArray)))
print('lantitudeArray'+str(np.shape(lantitudeArray)))
print('longitudeArray'+str(np.shape(longitudeArray)))
print('elevationArray'+str(np.shape(elevationArray)))
print('moistureArray'+str(np.shape(moistureArray)))
print('spatialMetaDataArray'+str(np.shape(spatialMetaDataArray)))


airTempArray(288, 51, 60)
netLongRadiantionFluxArray(288, 51, 60)
shortwaveRadiationArray(288, 51, 60)
precipitationArray(288, 51, 60)
NPPArray(288, 51, 60)
landSeaMaskArray(51, 60)
lantitudeArray(51, 60)
longitudeArray(51, 60)
elevationArray(51, 60)
moistureArray(51, 60)
spatialMetaDataArray()


In [7]:
import pandas as pd
landSeaMaskArray_flatten = landSeaMaskArray.reshape(51*60)
lantitudeArray_flatten = lantitudeArray.reshape(51*60)
longitudeArray_flatten = longitudeArray.reshape(51*60)
elevationArray_flatten = elevationArray.reshape(51*60)
moistureArray_flatten = moistureArray.reshape(51*60)
geo_attri = np.array(list(zip(landSeaMaskArray_flatten,lantitudeArray_flatten,longitudeArray_flatten,elevationArray_flatten,moistureArray_flatten))).repeat(repeats=5,axis=0)


In [8]:
geo_attri

array([[ 0.00000000e+00,  5.25000000e+00, -7.97500000e+01,
        -3.28700000e+03,  1.04999997e-01],
       [ 0.00000000e+00,  5.25000000e+00, -7.97500000e+01,
        -3.28700000e+03,  1.04999997e-01],
       [ 0.00000000e+00,  5.25000000e+00, -7.97500000e+01,
        -3.28700000e+03,  1.04999997e-01],
       ...,
       [ 1.00000000e+00, -1.97500000e+01, -5.02500000e+01,
         4.21000000e+02,  7.50000030e-02],
       [ 1.00000000e+00, -1.97500000e+01, -5.02500000e+01,
         4.21000000e+02,  7.50000030e-02],
       [ 1.00000000e+00, -1.97500000e+01, -5.02500000e+01,
         4.21000000e+02,  7.50000030e-02]])

In [9]:
airTempArray_flatten= airTempArray.reshape(288*51*60)
netLongRadiantionFluxArray_flatten= netLongRadiantionFluxArray.reshape(288*51*60)
shortwaveRadiationArray_flatten= shortwaveRadiationArray.reshape(288*51*60)
precipitationArray_flatten= precipitationArray.reshape(288*51*60)
NPPArray_flatten= NPPArray.reshape(288*51*60)
geo_data = np.array(list(zip(airTempArray_flatten,netLongRadiantionFluxArray_flatten,shortwaveRadiationArray_flatten,precipitationArray_flatten,NPPArray_flatten))).reshape(288,51*60*5)

C:\Users\new\AppData\Local\Temp\ipykernel_25664\1199548780.py:6: UserWarning: Warning: converting a masked element to nan.
  geo_data = np.array(list(zip(airTempArray_flatten,netLongRadiantionFluxArray_flatten,shortwaveRadiationArray_flatten,precipitationArray_flatten,NPPArray_flatten))).reshape(288,51*60*5)


In [13]:
time_depend = pd.DataFrame(geo_data)

In [14]:
static_depend = pd.DataFrame(geo_attri).stack().unstack(0)

In [15]:
output_data_2 = pd.concat([static_depend,time_depend],axis=0)

In [17]:
output_data_2.columns = pd.MultiIndex.from_product([['a'],output_data_2.columns])

In [21]:
pd.DataFrame(columns =pd.MultiIndex.from_product([['a','b'],['c','d','e']]))


Empty DataFrame
Columns: [(a, c), (a, d), (a, e), (b, c), (b, d), (b, e)]
Index: []

In [18]:
output_data_2

a                                                   \
           0            1            2            3         4       
0       0.000000     0.000000     0.000000     0.000000     0.000   
1       5.250000     5.250000     5.250000     5.250000     5.250   
2     -79.750000   -79.750000   -79.750000   -79.750000   -79.750   
3   -3287.000000 -3287.000000 -3287.000000 -3287.000000 -3287.000   
4       0.105000     0.105000     0.105000     0.105000     0.105   
..           ...          ...          ...          ...       ...   
283   300.299530   -42.329102   225.392441     0.000116       NaN   
284   300.312042   -36.887695   223.165619     0.000133       NaN   
285   300.087555   -38.923828   212.025772     0.000153       NaN   
286   299.646790   -39.163086   202.380325     0.000132       NaN   
287   300.570953   -42.825195   215.558350     0.000064       NaN   

                                                                   ...  \
           5            6            7            8         9      ...   
0       0.000000     0.000000     0.000000     0.000000     0.000  ...   
1       5.250000     5.250000     5.250000     5.250000     5.250  ...   
2     -79.250000   -79.250000   -79.250000   -79.250000   -79.250  ...   
3   -3317.000000 -3317.000000 -3317.000000 -3317.000000 -3317.000  ...   
4       0.105000     0.105000     0.105000     0.105000     0.105  ...   
..           ...          ...          ...          ...       ...  ...   
283   300.041321   -43.046875   229.956360     0.000097       NaN  ...   
284   299.840637   -36.570312   230.689056     0.000066       NaN  ...   
285   299.911194   -41.570312   233.069580     0.000100       NaN  ...   
286   299.572754   -39.968750   212.226547     0.000126       NaN  ...   
287   300.228210   -40.906250   204.465103     0.000085       NaN  ...   

                                                                             \
          15290       15291       15292       15293       15294       15295   
0      1.000000    1.000000    1.000000    1.000000    1.000000    1.000000   
1    -19.750000  -19.750000  -19.750000  -19.750000  -19.750000  -19.750000   
2    -50.750000  -50.750000  -50.750000  -50.750000  -50.750000  -50.250000   
3    391.000000  391.000000  391.000000  391.000000  391.000000  421.000000   
4      0.075000    0.075000    0.075000    0.075000    0.075000    0.075000   
..          ...         ...         ...         ...         ...         ...   
283  291.485840  -67.195312  203.676865    0.000014    0.985850  291.485840   
284  296.544495  -80.414062  242.964050    0.000016    0.801262  296.544495   
285  301.662415  -77.945312  286.565033    0.000041    1.495940  301.662415   
286  300.805542  -73.203125  294.562408    0.000060    3.435606  300.805542   
287  302.103027  -62.742188  295.242523    0.000042    4.009297  302.103027   

                                                     
          15296       15297       15298       15299  
0      1.000000    1.000000    1.000000    1.000000  
1    -19.750000  -19.750000  -19.750000  -19.750000  
2    -50.250000  -50.250000  -50.250000  -50.250000  
3    421.000000  421.000000  421.000000  421.000000  
4      0.075000    0.075000    0.075000    0.075000  
..          ...         ...         ...         ...  
283  -67.195312  203.676865    0.000014    0.985850  
284  -80.414062  242.964050    0.000016    0.801262  
285  -77.945312  286.565033    0.000041    1.495940  
286  -73.203125  294.562408    0.000060    3.435606  
287  -62.742188  295.242523    0.000042    4.009297  

[293 rows x 15300 columns]

In [22]:
geo_data = pd.DataFrame(geo_data).stack().unstack(0)

In [23]:
df = pd.DataFrame(geo_attri)

In [24]:
df = df.rename(columns={0: 'landSea', 1: 'lantitude', 2: 'longitude', 3: 'elevation', 4: 'moisture'})
# df['geo_attri'] = df[[0,1,2,3,4]].values.tolist()
# df = df.drop([0,1,2,3,4],axis = 1)

In [25]:
output_data = pd.concat([df,geo_data],axis=1)
# output_data_grouby = output_data.groupby(['landSea','lantitude','longitude','elevation','moisture'])
# output_data_grouby

In [30]:
output_data.set_index(['landSea','lantitude','longitude','elevation','moisture']).transpose()

MultiIndex([(0.0,   5.25, -79.75, -3287.0, 0.10499999672174454),
            (0.0,   5.25, -79.75, -3287.0, 0.10499999672174454),
            (0.0,   5.25, -79.75, -3287.0, 0.10499999672174454),
            (0.0,   5.25, -79.75, -3287.0, 0.10499999672174454),
            (0.0,   5.25, -79.75, -3287.0, 0.10499999672174454),
            (0.0,   5.25, -79.25, -3317.0, 0.10499999672174454),
            (0.0,   5.25, -79.25, -3317.0, 0.10499999672174454),
            (0.0,   5.25, -79.25, -3317.0, 0.10499999672174454),
            (0.0,   5.25, -79.25, -3317.0, 0.10499999672174454),
            (0.0,   5.25, -79.25, -3317.0, 0.10499999672174454),
            ...
            (1.0, -19.75, -50.75,   391.0, 0.07500000298023224),
            (1.0, -19.75, -50.75,   391.0, 0.07500000298023224),
            (1.0, -19.75, -50.75,   391.0, 0.07500000298023224),
            (1.0, -19.75, -50.75,   391.0, 0.07500000298023224),
            (1.0, -19.75, -50.75,   391.0, 0.07500000298023224),
         

In [8]:
print(airTempDataset)
for dm in airTempDataset.dimensions.values() :
    print(dm)

print(airTempDataset['var1'])



print(np.shape(airTempArray))

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    CDI: Climate Data Interface version 1.9.8 (https://mpimet.mpg.de/cdi)
    Conventions: CF-1.6
    history: Tue Jun 14 10:29:59 2022: ncks -4 -L4 -O tas.nc tas.nc
Wed Jun 08 09:48:43 2022: cdo -r -f nc copy tas.srv tas.nc
    CDO: Climate Data Operators version 1.9.8 (https://mpimet.mpg.de/cdo)
    NCO: netCDF Operators version 4.9.1 (Homepage = http://nco.sf.net, Code = http://github.com/nco/nco)
    dimensions(sizes): lev(1), time(288), y(51), x(60)
    variables(dimensions): float64 lev(lev), float64 time(time), float64 var1(time, lev, y, x)
    groups: 
<class 'netCDF4._netCDF4.Dimension'>: name = 'lev', size = 1
<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 288
<class 'netCDF4._netCDF4.Dimension'>: name = 'y', size = 51
<class 'netCDF4._netCDF4.Dimension'>: name = 'x', size = 60
<class 'netCDF4._netCDF4.Variable'>
float64 var1(time, lev, y, x)
unlimited dimensions: ti

In [10]:
netLongRadiantionFluxDataSet.dimensions.values()

dict_values([<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 288, <class 'netCDF4._netCDF4.Dimension'>: name = 'y', size = 51, <class 'netCDF4._netCDF4.Dimension'>: name = 'x', size = 60])

In [9]:
# print(netLongRadiantionFluxDataSet)
for dm in netLongRadiantionFluxDataSet.dimensions.values() :
    print(dm)

print(netLongRadiantionFluxDataSet['var1'])

<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 288
<class 'netCDF4._netCDF4.Dimension'>: name = 'y', size = 51
<class 'netCDF4._netCDF4.Dimension'>: name = 'x', size = 60
<class 'netCDF4._netCDF4.Variable'>
float64 var1(time, y, x)
unlimited dimensions: time
current shape = (288, 51, 60)
filling on, default _FillValue of 9.969209968386869e+36 used


In [1]:
# print(shortwaveRadiationDataset)
for dm in shortwaveRadiationDataset.dimensions.values() :
    print(dm)

NameError: name 'shortwaveRadiationDataset' is not defined

In [19]:
?precipitationDataSet

Type:           Dataset
String form:   
<class 'netCDF4._netCDF4.Dataset'>
           root group (NETCDF4 data model, file format HDF5):
           CDI: Cl <...> y(51), x(60)
           variables(dimensions): float64 time(time), float64 var1(time, y, x)
           groups:
File:           d:\ana\lib\site-packages\netcdf4\_netcdf4.cp39-win_amd64.pyd
Docstring:     
A netCDF `Dataset` is a collection of dimensions, groups, variables and
attributes. Together they describe the meaning of data and relations among
data fields stored in a netCDF file. See `Dataset.__init__` for more
details.

A list of attribute names corresponding to global netCDF attributes
defined for the `Dataset` can be obtained with the
`Dataset.ncattrs` method.
These attributes can be created by assigning to an attribute of the
`Dataset` instance. A dictionary containing all the netCDF attribute
name/value pairs is provided by the `__dict__` attribute of a
`Dataset` instance.

The following class variables are read-only

In [14]:
precipitationArray

masked_array(
  data=[[[1.48286263e-05, 2.63395868e-05, 2.63395868e-05, ...,
          8.80149819e-05, 8.80149819e-05, 8.80149819e-05],
         [1.48286263e-05, 2.63395868e-05, 2.63395868e-05, ...,
          8.80149819e-05, 8.80149819e-05, 8.80149819e-05],
         [4.74545996e-05, 9.03675827e-05, 9.03675827e-05, ...,
          1.44376652e-04, 1.44376652e-04, 1.44376652e-04],
         ...,
         [3.12384304e-06, 4.36076016e-06, 4.36076016e-06, ...,
          6.68623688e-05, 6.68623688e-05, 6.68623688e-05],
         [2.75390357e-06, 3.64959965e-06, 3.64959965e-06, ...,
          7.64823999e-05, 7.64823999e-05, 7.64823999e-05],
         [1.64238872e-06, 1.51285712e-06, 1.51285712e-06, ...,
          1.05386614e-04, 1.05386614e-04, 1.05386614e-04]],

        [[1.91101735e-05, 2.50428966e-05, 2.50428966e-05, ...,
          8.86672642e-05, 8.86672642e-05, 8.86672642e-05],
         [1.91101735e-05, 2.50428966e-05, 2.50428966e-05, ...,
          8.86672642e-05, 8.86672642e-05, 8.86672642e

In [12]:
# print(precipitationDataSet)
for dm in precipitationDataSet.dimensions.values() :
    print(dm)

<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 288
<class 'netCDF4._netCDF4.Dimension'>: name = 'y', size = 51
<class 'netCDF4._netCDF4.Dimension'>: name = 'x', size = 60


In [20]:
# print(outputDataset)
# for dm in outputDataset.dimensions.values() :
#     print(dm)

# print(outputDataset['NPP'])

{'AR': <class 'netCDF4._netCDF4.Variable'>
 float32 AR(time, y, x)
     long_name: Autotrophic Respiration
     units: gC/m^2/day
     code: 5310
     _FillValue: -9e+33
     missing_value: -9e+33
 unlimited dimensions: time
 current shape = (288, 51, 60)
 filling on,
 'ASSMC': <class 'netCDF4._netCDF4.Variable'>
 float32 ASSMC(time, y, x)
     long_name: Storage carbon pool
     units: gC/m^2
     code: 5373
     _FillValue: -9e+33
     missing_value: -9e+33
 unlimited dimensions: time
 current shape = (288, 51, 60)
 filling on,
 'ASSMC_LOSS': <class 'netCDF4._netCDF4.Variable'>
 float32 ASSMC_LOSS(time, y, x)
     long_name: Assimilate carbon loss
     units: gC/m^2/day
     code: 5616
     _FillValue: -9e+33
     missing_value: -9e+33
 unlimited dimensions: time
 current shape = (288, 51, 60)
 filling on,
 'BM': <class 'netCDF4._netCDF4.Variable'>
 float32 BM(time, y, x)
     long_name: Biomass
     units: gC/m^2
     code: 5304
     _FillValue: -9e+33
     missing_value: -9e+33
 un

In [12]:
print(np.shape(lantitudeArray))

(51, 60)
